In [16]:
import pandas as pd   
import warnings; warnings.simplefilter('ignore')

## Dual Simplex Method

In [17]:


objective="max"
m=3#no of equations
n=2#no of variables
a=[[-3,-1,'<='],[-4,-3,"<="],[-1,-2,"<="]]
b=[-3,-6,-3]
Coeff=[-2,-1]

In [18]:
def convert_to_maximization_type(a,b,Coeff,m,n,objective):
    if objective != 'max':
        Coeff=[-Coeff[i] for i in range(n)]
    for i in range(m):
        if a[i][-1] != '<=':
            for j in range(n):
                a[i][j]=-a[i][j]
            b[i]=-b[i]
    return a,b,Coeff,m,n
                
                
    

In [24]:


#1
def initialize_simplex_table(a, b, m, n, Coeff,objective):
    convert_dict={'x'+str(i+1):float for i in range(n)}
    jconvert_dict={'x'+str(i+1):float for i in range(n)}
    a,b,Coeff,m,n=convert_to_maximization_type(a,b,Coeff,m,n,objective)
    Cj_dict={'x'+str(i+1):[Coeff[i]] for i in range(n)}
    Zj_dict={'x'+str(i+1):[0] for i in range(n)}
    for i in range(m):
        Cj_dict['s'+str(i+1)]=[0]
        Zj_dict['s'+str(i+1)]=[0]
        convert_dict['s'+str(i+1)]=float
        jconvert_dict['s'+str(i+1)]=float
    Cj_dict['sol']=0
    Zj_dict['sol']=0   
    convert_dict['sol']=float
    jconvert_dict['sol']=float
    #Cj_dict.extend({0 for i in range(m)])
    #Zj=[0 for i in range(m+n)]
    data = {'CBi':[0 for i in range(m)]} 
    convert_dict['CBi']=float
    data['basic_var']=['s'+str(i+1) for i in range(m)]
    convert_dict['basic_var']=str
    for i in range(n):
        data['x'+str(i+1)]=[a[j][i] for j in range(m)]
    #augmenting data with slack variables
    for i in range(m):
        data['s'+str(i+1)]=[float(i==j) for j in range(m)]
    data['sol']=[b[j] for j in range(m)]
    #data['ratio']=[0 for j in range(m)]
    #convert_dict['ratio']=float
    df = pd.DataFrame(data, index=[str(i+1) for i in range(m)]) 
    Zj = pd.DataFrame(Zj_dict, index=['Zj']) 
    Cj = pd.DataFrame(Cj_dict, index=['Cj'])
    
    df = df.astype(convert_dict) 
    
    Zj = Zj.astype(jconvert_dict)
    Cj = Cj.astype(jconvert_dict)
    return df,Cj,Zj
  

In [25]:
#2 
def print_table(df, Cj, Zj):
    print("simplex table:")
    frames = [df, Cj, Zj]
    result = pd.concat(frames)    
    print(result)
#     print("Cj's: ",Cj)
    
#     print("Zj's: ",Zj)
    
    
  
 
df,Cj,Zj = initialize_simplex_table(a, b, m, n, Coeff,objective)
print_table(df,Cj,Zj)

simplex table:
    CBi basic_var   x1   x2   s1   s2   s3  sol
1   0.0        s1 -3.0 -1.0  1.0  0.0  0.0 -3.0
2   0.0        s2 -4.0 -3.0  0.0  1.0  0.0 -6.0
3   0.0        s3 -1.0 -2.0  0.0  0.0  1.0 -3.0
Cj  NaN       NaN -2.0 -1.0  0.0  0.0  0.0  0.0
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0


In [26]:
#3
def Zj_Cj(Cj, Zj):
    l=['x'+str(i+1) for i in range(n)]
    l.extend(['s'+str(i+1) for i in range(m)])
    return  [Zj[i]['Zj']-Cj[i]['Cj'] for i in l]


In [27]:
#4
def col_name(i,n):
    if (i+1)<=n:
        return 'x'+str(i+1)
    else:
        return 's'+str(i+1-n)
    
def get_key_column(zj_cj,df, m, n, key_row):
    val_max=-999999999
    idx_max='x1'  
    for i in range(m+n):
        c_name=col_name(i,n)
        ratio=zj_cj[i]/df[c_name][key_row]
        if df[c_name][key_row]<0:
            if val_max<ratio:
                val_max=ratio
                idx_max=c_name
    
    return idx_max
    


def get_key_row(df, m):     
    val_min=0
    idx_min=0
    for i in range(m):
        if val_min>df['sol'][i]:
            val_min=df['sol'][i]
            idx_min=i
    return df, str(idx_min+1), val_min




In [28]:
print("_____output_________")
zj_cj=Zj_Cj(Cj, Zj)
df, key_row, min_ratio = get_key_row(df, m)
key_col=get_key_column(zj_cj,df, m, n,key_row)

print(df)
print()
print("Key column is " )
print(df[key_col])
print()
print("Minimum ratio is ", min_ratio)
print()
print("Key row is ")
print(df.loc[key_row])
print()
print("Key element is ", df[key_col][key_row])


_____output_________
   CBi basic_var   x1   x2   s1   s2   s3  sol
1  0.0        s1 -3.0 -1.0  1.0  0.0  0.0 -3.0
2  0.0        s2 -4.0 -3.0  0.0  1.0  0.0 -6.0
3  0.0        s3 -1.0 -2.0  0.0  0.0  1.0 -3.0

Key column is 
1   -1.0
2   -3.0
3   -2.0
Name: x2, dtype: float64

Minimum ratio is  -6.0

Key row is 
CBi           0
basic_var    s2
x1           -4
x2           -3
s1            0
s2            1
s3            0
sol          -6
Name: 2, dtype: object

Key element is  -3.0


In [29]:
def modify_table(key_row, key_col, Cj, Zj,df):
    df_new = df.copy(deep=True)
    Zj_new = Zj.copy(deep=True)
    col_len=len(df[key_col])
    row_len=len(df.loc[key_row])
    p=key_elem=df[key_col][key_row]
    entering_var=key_col
    leaving_var=df['basic_var'][key_row]
    print("Entering variable is ", entering_var)
    print("Leaving variable is ", leaving_var)
    df_new['basic_var'][key_row]=entering_var
    
    
    col=['x'+str(i+1) for i in range(n)]
    col.extend(['s'+str(i+1) for i in range(m)])
    col.append('sol')
    row=list(df.index)
    
    for j in col:
        r=df[j][key_row]              
        for i in row:    
            if j==key_col:
                df_new[j][i]=(-df[j][i])/p                
            elif i==key_row:
                df_new[j][i]=df[j][i]/p               
            else:
                s=df[j][i]
                q=df[key_col][i] 
                df_new[j][i]=(p*s-q*r)/p
    #print(1/p)
    df_new[key_col][key_row]=1/p
    #df_new[key_col]=df[]
    df_new[leaving_var]  = df_new[entering_var]    
    df_new[entering_var] = 0
    df_new[entering_var][key_row]=1
    
    #swap the coefficients value
    df_new['CBi'][key_row]=Cj[key_col]['Cj']
    
    #modify zj
    for j in col:
        Zj_new[j]['Zj']=sum([df_new[j][i]*df_new['CBi'][i] for i in row]) 
        #Zj_new[j]['Zj']=16.0*0.05
    return df_new,Cj,Zj_new
    
    
    
    
    
    
    
    
    

In [30]:
df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df)
print_table(df_new,Cj,Zj_new)


Entering variable is  x2
Leaving variable is  s2
simplex table:
    CBi basic_var        x1   x2   s1        s2   s3  sol
1   0.0        s1 -1.666667  0.0  1.0 -0.333333 -0.0 -1.0
2  -1.0        x2  1.333333  1.0 -0.0 -0.333333 -0.0  2.0
3   0.0        s3  1.666667  0.0 -0.0 -0.666667  1.0  1.0
Cj  NaN       NaN -2.000000 -1.0  0.0  0.000000  0.0  0.0
Zj  NaN       NaN -1.333333 -1.0  0.0  0.333333  0.0 -2.0


In [31]:
#Check is optimal solution is reached or not
def is_feasible(df):
    for val in df['sol']:
        if val<0:
            print("Lets proceed to next iteration")
            return False
    else:   
        print("Optimality is reached")
        return True
    
        

In [34]:
def simplex_method(a, b, m, n, Coeff,objective):    
    df,Cj,Zj = initialize_simplex_table(a, b, m, n, Coeff,objective)
    zj_cj=Zj_Cj(Cj, Zj)
    df, key_row, min_ratio = get_key_row(df, m)
    key_col=get_key_column(zj_cj,df, m, n,key_row)
    print_table(df,Cj,Zj)
    i=0
    while not is_feasible(df) and i<10:
        print("_______________________________________________________")
        if df[key_col][key_row]==0:
            print("Problem is unbounded")
            return
        df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df)
        
        df=df_new
        Zj=Zj_new
        
        if objective=="max":
            print("value of objective function is ",Zj['sol']['Zj'])
        else:
            print("value of objective function is ",-Zj['sol']['Zj'])
        
        zj_cj=Zj_Cj(Cj, Zj)
        df, key_row, min_ratio = get_key_row(df, m)
        key_col=get_key_column(zj_cj,df, m, n,key_row)  
        print_table(df,Cj,Zj)
        i+=1
        
    
    

In [35]:
simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1   x2   s1   s2   s3  sol
1   0.0        s1 -3.0 -1.0  1.0  0.0  0.0 -3.0
2   0.0        s2 -4.0 -3.0  0.0  1.0  0.0 -6.0
3   0.0        s3 -1.0 -2.0  0.0  0.0  1.0 -3.0
Cj  NaN       NaN -2.0 -1.0  0.0  0.0  0.0  0.0
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x2
Leaving variable is  s2
value of objective function is  -2.0
simplex table:
    CBi basic_var        x1   x2   s1        s2   s3  sol
1   0.0        s1 -1.666667  0.0  1.0 -0.333333 -0.0 -1.0
2  -1.0        x2  1.333333  1.0 -0.0 -0.333333 -0.0  2.0
3   0.0        s3  1.666667  0.0 -0.0 -0.666667  1.0  1.0
Cj  NaN       NaN -2.000000 -1.0  0.0  0.000000  0.0  0.0
Zj  NaN       NaN -1.333333 -1.0  0.0  0.333333  0.0 -2.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s1
value of objective functi

In [36]:
#a
objective="min"
m=4#no of equations
n=2#no of variables
a=[[1,0,'>='],[0,1,">="],[2,1,">="],[1,1,">="]]
b=[2.5,6,17,12]
Coeff=[20,16]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var    x1    x2   s1   s2   s3   s4   sol
1   0.0        s1  -1.0   0.0  1.0  0.0  0.0  0.0  -2.5
2   0.0        s2   0.0  -1.0  0.0  1.0  0.0  0.0  -6.0
3   0.0        s3  -2.0  -1.0  0.0  0.0  1.0  0.0 -17.0
4   0.0        s4  -1.0  -1.0  0.0  0.0  0.0  1.0 -12.0
Cj  NaN       NaN -20.0 -16.0  0.0  0.0  0.0  0.0   0.0
Zj  NaN       NaN   0.0   0.0  0.0  0.0  0.0  0.0   0.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s3
value of objective function is  170.0
simplex table:
     CBi basic_var    x1    x2   s1   s2    s3   s4    sol
1    0.0        s1   0.0   0.5  1.0 -0.0  -0.5 -0.0    6.0
2    0.0        s2   0.0  -1.0  0.0  1.0   0.0  0.0   -6.0
3  -20.0        x1   1.0   0.5 -0.0 -0.0  -0.5 -0.0    8.5
4    0.0        s4   0.0  -0.5 -0.0 -0.0  -0.5  1.0   -3.5
Cj   NaN       NaN -20.0 -16.0  0.0  0.0   0.0  0.0    0.0
Zj   NaN       NaN -20.0 -10.0  0.0  0.0  10.0  0.

In [37]:
#b
objective="min"
m=2#no of equations
n=3#no of variables
a=[[1,1,0,'>='],[2,0,1,"<="]]
b=[2,5]
Coeff=[4,8,3]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1   x2   x3   s1   s2  sol
1   0.0        s1 -1.0 -1.0  0.0  1.0  0.0 -2.0
2   0.0        s2  2.0  0.0  1.0  0.0  1.0  5.0
Cj  NaN       NaN -4.0 -8.0 -3.0  0.0  0.0  0.0
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s1
value of objective function is  8.0
simplex table:
    CBi basic_var   x1   x2   x3   s1   s2  sol
1  -4.0        x1  1.0  1.0 -0.0 -1.0 -0.0  2.0
2   0.0        s2  0.0 -2.0  1.0  2.0  1.0  1.0
Cj  NaN       NaN -4.0 -8.0 -3.0  0.0  0.0  0.0
Zj  NaN       NaN -4.0 -4.0  0.0  4.0  0.0 -8.0
Optimality is reached


In [38]:
#c
objective="max"
m=3#no of equations
n=4#no of variables
a=[[10,5,25,3,'<='],[12,4,12,1,"<="],[7,0,0,1,"<="]]
b=[50,48,35]
Coeff=[15,6,9,2]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var    x1   x2    x3   x4   s1   s2   s3   sol
1   0.0        s1  10.0  5.0  25.0  3.0  1.0  0.0  0.0  50.0
2   0.0        s2  12.0  4.0  12.0  1.0  0.0  1.0  0.0  48.0
3   0.0        s3   7.0  0.0   0.0  1.0  0.0  0.0  1.0  35.0
Cj  NaN       NaN  15.0  6.0   9.0  2.0  0.0  0.0  0.0   0.0
Zj  NaN       NaN   0.0  0.0   0.0  0.0  0.0  0.0  0.0   0.0
Optimality is reached


In [39]:
#d
objective="max"
m=3#no of equations
n=3#no of variables
a=[[2,2,-1,'>='],[3,-4,0,"<="],[0,1,3,"<="]]
b=[2,3,3]
Coeff=[5,-2,3]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1   x2   x3   s1   s2   s3  sol
1   0.0        s1 -2.0 -2.0  1.0  1.0  0.0  0.0 -2.0
2   0.0        s2  3.0 -4.0  0.0  0.0  1.0  0.0  3.0
3   0.0        s3  0.0  1.0  3.0  0.0  0.0  1.0  3.0
Cj  NaN       NaN  5.0 -2.0  3.0  0.0  0.0  0.0  0.0
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s1
value of objective function is  5.0
simplex table:
    CBi basic_var   x1   x2   x3   s1   s2   s3  sol
1   5.0        x1  1.0  1.0 -0.5 -0.5 -0.0 -0.0  1.0
2   0.0        s2  0.0 -7.0  1.5  1.5  1.0  0.0 -0.0
3   0.0        s3  0.0  1.0  3.0  0.0  0.0  1.0  3.0
Cj  NaN       NaN  5.0 -2.0  3.0  0.0  0.0  0.0  0.0
Zj  NaN       NaN  5.0  5.0 -2.5 -2.5  0.0  0.0  5.0
Optimality is reached


In [40]:
#e
objective="max"
m=3#no of equations
n=3#no of variables
a=[[1,1,1,'<='],[2,1,-1,">="],[0,-1,1,">="]]
b=[40,10,10]
Coeff=[2,3,1]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1   x2   x3   s1   s2   s3   sol
1   0.0        s1  1.0  1.0  1.0  1.0  0.0  0.0  40.0
2   0.0        s2 -2.0 -1.0  1.0  0.0  1.0  0.0 -10.0
3   0.0        s3  0.0  1.0 -1.0  0.0  0.0  1.0 -10.0
Cj  NaN       NaN  2.0  3.0  1.0  0.0  0.0  0.0   0.0
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0   0.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x2
Leaving variable is  s2
value of objective function is  30.0
simplex table:
    CBi basic_var   x1   x2   x3   s1   s2   s3   sol
1   0.0        s1 -1.0  0.0  2.0  1.0  1.0  0.0  30.0
2   3.0        x2  2.0  1.0 -1.0 -0.0 -1.0 -0.0  10.0
3   0.0        s3 -2.0  0.0 -0.0  0.0  1.0  1.0 -20.0
Cj  NaN       NaN  2.0  3.0  1.0  0.0  0.0  0.0   0.0
Zj  NaN       NaN  6.0  3.0 -3.0  0.0 -3.0  0.0  30.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s3
value of obje

In [41]:
#f
objective="max"
m=3#no of equations
n=2#no of variables
a=[[3,2,'>='],[1,4,">="],[1,1,"<="]]
b=[3,4,5]
Coeff=[5,8]

simplex_method(a, b, m, n, Coeff,objective)

simplex table:
    CBi basic_var   x1   x2   s1   s2   s3  sol
1   0.0        s1 -3.0 -2.0  1.0  0.0  0.0 -3.0
2   0.0        s2 -1.0 -4.0  0.0  1.0  0.0 -4.0
3   0.0        s3  1.0  1.0  0.0  0.0  1.0  5.0
Cj  NaN       NaN  5.0  8.0  0.0  0.0  0.0  0.0
Zj  NaN       NaN  0.0  0.0  0.0  0.0  0.0  0.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s2
value of objective function is  20.0
simplex table:
    CBi basic_var   x1    x2   s1   s2   s3   sol
1   0.0        s1  0.0  10.0  1.0 -3.0 -0.0   9.0
2   5.0        x1  1.0   4.0 -0.0 -1.0 -0.0   4.0
3   0.0        s3  0.0  -3.0  0.0  1.0  1.0   1.0
Cj  NaN       NaN  5.0   8.0  0.0  0.0  0.0   0.0
Zj  NaN       NaN  5.0  20.0  0.0 -5.0  0.0  20.0
Optimality is reached


## Menu Program

### Input: 
<ol>
    <li>First line will contain number of test cases t</li>
    <li>Format for next t is as follows</li>
    <li>
        <ol>
            <li> Objective of the LPP: min or max</li>
            <li>2 spaced integers m,n denoting number of equations and number of variables respectively</li>
            <li>For next m lines, each line contains n integers (i.e) ai_1 to ai_n followed by the type of inequality &le; or &ge;</li>
            <li>m spaced integers b1 to bm</li>
            <li>n spaced integers denoting coefficients of objective function</li>
        </ol>
    </li>
</ol>

### output:
<ol>
    <li>List of simplex tables at each iteration consisting of <br/>(a) initial table <br/> (b) List of basic and non-basic variables for ith iteration <br/>(c) table of ith iteration <br/>(f) optimal solution </li>
</ol>
    
    
                
                

In [45]:
#program to take input
def input_ai(val):
    if not (val == "<=" or val == ">="):
        return float(val)
    else:
        return val

t=int(input())
for i in range(t):
    print("Enter input in above format:")
    objective=input()
    m,n = map(int, input().split())
    a=[]
    for j in range(m):
        ai=list(map(input_ai, input().split()))
        a.append(ai)
    b=list(map(float, input().split()))
    Coeff=list(map(float, input().split()))
    print("Here is the output:")
    simplex_method(a, b, m, n, Coeff,objective)
    
        
        

 1
 min
 4 2
 1 0 >=
 0 1 >=
 2 1 >=
 1 1 >=
 2.5 6 17 12
 20 16


simplex table:
    CBi basic_var    x1    x2   s1   s2   s3   s4   sol
1   0.0        s1  -1.0  -0.0  1.0  0.0  0.0  0.0  -2.5
2   0.0        s2  -0.0  -1.0  0.0  1.0  0.0  0.0  -6.0
3   0.0        s3  -2.0  -1.0  0.0  0.0  1.0  0.0 -17.0
4   0.0        s4  -1.0  -1.0  0.0  0.0  0.0  1.0 -12.0
Cj  NaN       NaN -20.0 -16.0  0.0  0.0  0.0  0.0   0.0
Zj  NaN       NaN   0.0   0.0  0.0  0.0  0.0  0.0   0.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  s3
value of objective function is  170.0
simplex table:
     CBi basic_var    x1    x2   s1   s2    s3   s4    sol
1    0.0        s1   0.0   0.5  1.0 -0.0  -0.5 -0.0    6.0
2    0.0        s2   0.0  -1.0 -0.0  1.0  -0.0 -0.0   -6.0
3  -20.0        x1   1.0   0.5 -0.0 -0.0  -0.5 -0.0    8.5
4    0.0        s4   0.0  -0.5 -0.0 -0.0  -0.5  1.0   -3.5
Cj   NaN       NaN -20.0 -16.0  0.0  0.0   0.0  0.0    0.0
Zj   NaN       NaN -20.0 -10.0  0.0  0.0  10.0  0.